# 04b — Fine-tuning with Back-Translated Data

Repeat the fine-tuning process from `03_finetune.ipynb`,  
but uses the merged dataset `train_plus_bt.tsv`, which combines:
- Original parallel data (`train.tsv`)
- Synthetic back-translated pairs generated in `04_pivot_bt_eval.ipynb`

Goal: Assess whether data augmentation via back-translation improves BLEU and chrF2 scores.

### Fine-tuning with Back-Translated Data

This command runs the same `finetune.py` training script, but this time uses the augmented dataset that includes synthetic pairs generated through back-translation (BT).

In [1]:
!python ../src/train/finetune.py \
  --train ../data/processed/train_plus_bt.tsv \
  --dev ../data/processed/dev.tsv \
  --out ../experiments/finetune_bt

Resolved codes → src: 'ceb_Latn' as 'ceb_Latn' (id=256035) | tgt: 'tgl_Latn' as 'tgl_Latn' (id=256174)
Train samples: 42,851 | Dev samples: 2,930
{'loss': 2.4145, 'grad_norm': 2.515625, 'learning_rate': 1.9908530894157178e-05, 'epoch': 0.01}
{'loss': 2.3917, 'grad_norm': 3.484375, 'learning_rate': 1.9815195071868583e-05, 'epoch': 0.02}
{'loss': 2.2988, 'grad_norm': 3.328125, 'learning_rate': 1.9721859249579992e-05, 'epoch': 0.03}
{'loss': 2.2238, 'grad_norm': 3.359375, 'learning_rate': 1.9628523427291398e-05, 'epoch': 0.04}
{'loss': 2.215, 'grad_norm': 2.59375, 'learning_rate': 1.95351876050028e-05, 'epoch': 0.05}
{'loss': 2.1846, 'grad_norm': 3.34375, 'learning_rate': 1.944185178271421e-05, 'epoch': 0.06}
{'loss': 2.1204, 'grad_norm': 3.296875, 'learning_rate': 1.9348515960425614e-05, 'epoch': 0.07}
{'loss': 2.1726, 'grad_norm': 4.0625, 'learning_rate': 1.925518013813702e-05, 'epoch': 0.07}
{'loss': 2.0889, 'grad_norm': 2.9375, 'learning_rate': 1.9161844315848422e-05, 'epoch': 0.08}
{

`torch_dtype` is deprecated! Use `dtype` instead!

Tokenizing train: 100%|██████████| 42851/42851 [00:22<00:00, 1889.03 examples/s]

Tokenizing dev: 100%|██████████| 2930/2930 [00:01<00:00, 1806.73 examples/s]
d:\OneDrive\Documents\My Learning Resource\University Courses\DLSU\2025-26\T1\CSC715M\assignments\mc02\src\train\finetune.py:239: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(

  0%|          | 50/10714 [00:20<1:08:08,  2.61it/s]
                                                    

  1%|          | 100/10714 [00:33<36:00,  4.91it/s]
                                                   

  1%|▏         | 150/10714 [00:46<33:06,  5.32it/s]
                                                   

  2%|▏         | 200/10714 [00:57<34:53,  5.02it/s]
                                                   

  2%|▏         | 250/10714 [01:08<36:06,  4.83it/s]
                

Similar to the fine-tuning processing in `03_finetune.ipynb`, this cell:
- Loads the **combined training corpus**:
  - `../data/processed/train_plus_bt.tsv` → includes both real parallel data and synthetic (back-translated) examples.  
  - `../data/processed/dev.tsv` → validation set for monitoring performance.
- Fine-tunes the multilingual **NLLB-200 distilled 600M** model using the same hyperparameters and language codes (`ceb_Latn → tgl_Latn`).
- The additional synthetic Tagalog–Cebuano examples help the model generalize better to unseen patterns and reduce overfitting to limited real data.

This run allows comparison between:
- The **standard fine-tuning** (real parallel data only), and  
- The **BT-augmented fine-tuning**, which includes synthetic examples to improve translation quality and domain robustness.

### Evaluate the fine-tuned + back-translation model on the test set

Runs the evaluation script on the model trained with **train_plus_bt.tsv** (real + synthetic pairs). It translates the test set and computes **BLEU** and **chrF2** using SacreBLEU.

In [2]:
!python ../src/eval/evaluate.py \
  --model_dir ../experiments/finetune_bt \
  --test_tsv ../data/processed/test.tsv \
  --out_json ../experiments/finetune_bt/metrics.json \
  --save_hyp ../experiments/finetune_bt/hyp.txt

{
  "BLEU": 30.08,
  "chrF2": 56.64,
  "ref_len": 85119,
  "sys_len": 89786,
  "sacrebleu_version": "2.5.1",
  "n_samples": 2750,
  "model_dir": "../experiments/finetune_bt",
  "codes": {
    "src": "ceb_Latn",
    "tgt": "tgl_Latn"
  },
  "decoding": {
    "beams": 5,
    "max_new_tokens": 200,
    "batch_size": 16
  }
}


`torch_dtype` is deprecated! Use `dtype` instead!

Translating:   0%|          | 0/172 [00:00<?, ?it/s]C:\Users\julia\AppData\Roaming\Python\Python312\site-packages\transformers\generation\utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(

Translating: 100%|██████████| 172/172 [11:04<00:00,  3.86s/it]
